






















































































# 🎨 Vanilla ComfyUI Colab

A clean, minimal setup for running ComfyUI in Google Colab.

---

## 📦 Install ComfyUI & Dependencies

This will install ComfyUI, all required dependencies, and ComfyUI Manager.

In [1]:
import os
from pathlib import Path

WORKSPACE = "/content/ComfyUI"

# ========================================
# 1. Clone ComfyUI
# ========================================
if not os.path.exists(WORKSPACE):
    print("📥 Cloning ComfyUI repository...")
    !git clone https://github.com/comfyanonymous/ComfyUI {WORKSPACE}
    print("✅ ComfyUI cloned successfully\n")
else:
    print("✅ ComfyUI directory already exists\n")

# Change to ComfyUI directory
%cd {WORKSPACE}

# Update ComfyUI
print("🔄 Updating ComfyUI...")
!git pull
print("✅ ComfyUI updated\n")

# ========================================
# 2. Install Dependencies
# ========================================
print("📦 Installing dependencies...\n")

# Upgrade pip
!pip install --upgrade pip -q

# Install PyTorch with CUDA support
print("⚡ Installing PyTorch with CUDA 12.1...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q

# Install core dependencies
print("📚 Installing core dependencies...")
!pip install -q \
    accelerate \
    einops \
    "safetensors>=0.4.2" \
    aiohttp \
    pyyaml \
    Pillow \
    scipy \
    tqdm \
    psutil \
    "tokenizers>=0.13.3" \
    sentencepiece \
    soundfile \
    "kornia>=0.7.1" \
    spandrel \
    torchsde \
    comfy_aimdo \
    av \
    comfy-kitchen \
    comfyui-workflow-templates \
    comfyui-embedded-docs

# Install transformers and huggingface-hub with compatible versions
print("🤗 Installing transformers and huggingface-hub...")
!pip install -q \
    "transformers>=4.45.0,<4.57.0" \
    "huggingface-hub>=0.23.0,<1.0"

# Install optional speedup packages
print("🚀 Installing optional packages...")
!pip install -q hf_transfer

print("✅ All dependencies installed successfully!\n")

# ========================================
# 3. Install ComfyUI Manager
# ========================================
manager_path = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"

if not os.path.exists(manager_path):
    print("📥 Installing ComfyUI Manager...")
    !git clone https://github.com/ltdrdata/ComfyUI-Manager {manager_path}
    print("✅ ComfyUI Manager installed\n")
else:
    print("🔄 Updating ComfyUI Manager...")
    !cd {manager_path} && git pull
    print("✅ ComfyUI Manager updated\n")

# ========================================
# 4. Verify Installation
# ========================================
import importlib.metadata as metadata

print("📋 Installed versions:\n")

packages = [
    "torch",
    "transformers",
    "huggingface-hub",
    "tokenizers",
    "safetensors"
]

for pkg in packages:
    try:
        version = metadata.version(pkg)
        print(f"  ✅ {pkg}: {version}")
    except:
        print(f"  ❌ {pkg}: not found")

print("\n" + "="*50)
print("🎉 Installation complete!")
print("="*50)
print("\n✅ You can now run the Launch cell below!")
print("="*50)

📥 Cloning ComfyUI repository...
Cloning into '/content/ComfyUI'...
remote: Enumerating objects: 31596, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 31596 (delta 52), reused 40 (delta 39), pack-reused 31512 (from 2)
Receiving objects: 100% (31596/31596), 71.26 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (21480/21480), done.
✅ ComfyUI cloned successfully

/content/ComfyUI
🔄 Updating ComfyUI...
Already up to date.
✅ ComfyUI updated

📦 Installing dependencies...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.0 MB/s eta 0:00:00
⚡ Installing PyTorch with CUDA 12.1...
📚 Installing core dependencies...
🤗 Installing transformers and huggingface-hub...
🚀 Installing optional packages...
✅ All dependencies installed successfully!

📥 Installing ComfyUI Manager...
Cloning into '/content/ComfyUI/custom_nodes/ComfyUI-Manager'...
remote: Enumerating objects: 28313, done.
remote: Counting objects: 100% (786/786), done.

---

# 📩Install Custom Models and Nodes

In [2]:
# ========================================
# Fresh Start: Download Models ONLY
# ========================================
import os
from pathlib import Path

print("="*50)
print("📦 Downloading Models")
print("="*50)

WORKSPACE = "/content/ComfyUI"
MODELS_DIR = f"{WORKSPACE}/models"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Models - Corrected paths and names for your workflow
MODELS = [
    "https://huggingface.co/misri/juggernautXL_juggXIByRundiffusion/resolve/main/juggernautXL_juggXIByRundiffusion.safetensors",
    "https://huggingface.co/skbhadra/ClearRealityV1/resolve/main/4x-ClearRealityV1.pth",
    "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors",
    "https://huggingface.co/outfly/face_yolov8m/resolve/main/face_yolov8m.pt"
]

for url in MODELS:
    filename = url.split('/')[-1].split('?')[0]

    if "juggernaut" in url.lower():
        model_dir = f"{MODELS_DIR}/checkpoints"
    elif "upscale" in url.lower() or filename.endswith('.pth'):
        model_dir = f"{MODELS_DIR}/upscale_models"
    elif "controlnet" in url.lower():
        model_dir = f"{MODELS_DIR}/controlnet"
        filename = "diffusion_pytorch_model_promax.safetensors" # Rename for workflow
    elif "yolo" in url.lower() or "face" in url.lower():
        model_dir = f"{MODELS_DIR}/ultralytics/bbox"
    else:
        model_dir = f"{MODELS_DIR}/checkpoints"

    os.makedirs(model_dir, exist_ok=True)
    filepath = os.path.join(model_dir, filename)

    if not os.path.exists(filepath):
        print(f"📥 Downloading {filename}...")
        !wget -q --show-progress -c "{url}" -O "{filepath}"
    else:
        print(f"⏭️  {filename} exists")

print("\n✅ Models are ready.")

📦 Downloading Models
📥 Downloading juggernautXL_juggXIByRundiffusion.safetensors...
/content/ComfyUI/mo 100%[===================>]   6.62G  90.0MB/s    in 86s     
📥 Downloading 4x-ClearRealityV1.pth...
/content/ComfyUI/mo 100%[===================>]   8.60M  12.3MB/s    in 0.7s    
📥 Downloading diffusion_pytorch_model_promax.safetensors...
/content/ComfyUI/mo 100%[===================>]   2.34G  73.3MB/s    in 35s     
📥 Downloading face_yolov8m.pt...
/content/ComfyUI/mo 100%[===================>]  49.62M  43.1MB/s    in 1.2s    

✅ Models are ready.


In [10]:
# 1. Install Ninja (required for C++ compilation)
!pip install ninja

# 2. Go to the exact folder containing the custom_rasterizer setup.py
%cd /content/ComfyUI/custom_nodes/comfyui-hunyuan3dwrapper/hy3dgen/texgen/custom_rasterizer

# 3. Install the module specifically from this folder
# We use --no-build-isolation to ensure it uses Colab's pre-installed PyTorch/CUDA
!pip install . --no-build-isolation

# 4. Return to the main ComfyUI directory
%cd /content/ComfyUI

/content/ComfyUI/custom_nodes/comfyui-hunyuan3dwrapper/hy3dgen/texgen/custom_rasterizer
Processing ./.
  Preparing metadata (pyproject.toml) ... done
  Created wheel for custom_rasterizer: filename=custom_rasterizer-0.1.0+torch290.cuda126-cp312-cp312-linux_x86_64.whl size=5145051 sha256=ef05a0b412b219831cc3a3c925943fe1e8a8e0a8c82372a9562e4e953beb6bdf
  Stored in directory: /tmp/pip-ephem-wheel-cache-rd5o_o8k/wheels/9e/13/6d/2b48e80ddb5a584d1fc8f16ffa5f56fb968588261c388b3f3c
Successfully built custom_rasterizer
/content/ComfyUI


---

# 🚀 Launch ComfyUI

**Run this cell AFTER restarting the runtime.**

In [ ]:
import threading
import time
import socket

def iframe_thread(port):
    """Wait for server to start and display in iframe"""
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    from google.colab import output
    print("\n🌐 ComfyUI is ready! Opening in iframe below...")
    print("💡 Tip: Click the link below to open in a new window\n")
    output.serve_kernel_port_as_iframe(port, height=1024)
    output.serve_kernel_port_as_window(port)

# Change to ComfyUI directory
%cd /content/ComfyUI

# Start iframe thread
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

print("🚀 Starting ComfyUI...\n")
print("⏳ Please wait for the interface to load below...\n")

# Launch ComfyUI
!python main.py --dont-print-server

/content/ComfyUI
🚀 Starting ComfyUI...

⏳ Please wait for the interface to load below...

[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-02-06 05:36:32.281
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/__manager/config.ini
** Log path: /content/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/rgthree-comfy
   4.6 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Found comfy_kitchen backend eager: {'available': True, 'disabled': False, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_

<IPython.core.display.Javascript object>

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

FETCH ComfyRegistry Data: 20/124
FETCH ComfyRegistry Data: 25/124
FETCH ComfyRegistry Data: 30/124
[DEPRECATION WARNING] Detected import of deprecated legacy API: /scripts/ui.js. This is likely caused by a custom node extension using outdated APIs. Please update your extensions or contact the extension author for an updated version.
[DEPRECATION WARNING] Detected import of deprecated legacy API: /extensions/core/groupNode.js. This is likely caused by a custom node extension using outdated APIs. Please update your extensions or contact the extension author for an updated version.
[DEPRECATION WARNING] Detected import of deprecated legacy API: /extensions/core/clipspace.js. This is likely caused by a custom node extension using outdated APIs. Please update your extensions or contact the extension author for an updated version.
[DEPRECATION WARNING] Detected import of deprecated legacy API: /scripts/ui/components/buttonGroup.js. This is likely caused by a custom node extension using outda

---

## 📝 Notes

- **First time setup**: The installation takes 5-10 minutes
- **Models**: Download models through the ComfyUI interface or manually to `/content/ComfyUI/models/`
- **Custom nodes**: Use ComfyUI Manager (if installed) to add custom nodes
- **Persistence**: Files are stored in the Colab runtime and will be lost when the session ends
- **Google Drive**: Enable the Google Drive option to save workflows and outputs

## 🐛 Troubleshooting

- **403 Error**: Check your browser settings or disable extensions that might block iframes
- **Server won't start**: Restart the runtime and try again
- **Missing models**: Download required models to the appropriate folder in `/content/ComfyUI/models/`

---

**Enjoy using ComfyUI! 🎨**